<a href="https://colab.research.google.com/github/trefftzc/cis677/blob/main/Basis_for_project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## A skeleton for the second programming project

The purpose of this project is to give you the chance to observe how using numba
will accelerate the execution of a regular python program.

The acceleration is the result of two different factors:
1. The use of compiled code
2. The use of the prange directive to accelerate the execution of a loop by dividing the iterations of a for statement across several threads of execution.

Unfortunately, the machines that Google makes available through COLAB do not have many cores, so the second step does not result in a noticeable acceleration on COLAB. If you are curious, you can replicate the experiments on your EOS account or on your own computer and you should observe better speedups.

You are provided two exact copies of the same sequential code.

Leave the first copy as it is, so that you can compare the execution times.

Follow the instructions below, to accelerate the second copy.

We start by creating several different test files.


## Test files

A complete graph (all nodes are connected to all other nodes) with four nodes.
The maximum independent set has size 1. It can be any node.

In [40]:
%%writefile k4.txt
4
0 1 1 1
1 0 1 1
1 1 0 1
1 1 1 0

Overwriting k4.txt


A graph with no edges and four nodes. The maximum independent set has four nodes $\{0,1,2,3\}$

In [41]:
%%writefile no_edges_4.txt
4
0 0 0 0
0 0 0 0
0 0 0 0
0 0 0 0

Overwriting no_edges_4.txt


A complete graph with sixteen nodes. Again, the maximum independent set has size 1 and it can be any node.

In [42]:
%%writefile k16.txt
16
0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1
1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1
1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0

Overwriting k16.txt


A complete graph with 20 nodes. Again, the maximum independent set has size 1 and it can be any node. Finding the maximum independent set for this graph will take more than five minutes with the pure python sequential version.

In [43]:
%%writefile k20.txt
20
0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0

Overwriting k20.txt


A complete graph with 24 nodes. Again, the maximum independent set has size 1 and it can be any node. Finding the maximum independent set for this graph will take more than five minutes with the pure python sequential version.

In [44]:
%%writefile k24.txt
24
0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0

Overwriting k24.txt


Do not modify this file. This is the sequential version that will be used for comparison purposes.

In [45]:
%%writefile original_python.py
import sys
import time
import numpy as np

def read_adjacency_matrix(file_name):
  file_object = open(file_name, "r")
  # Input the number of rows and columns
  size = int(file_object.readline())
  rows = size
  cols = size
  # Initialize an empty matrix
  matrix = []

  # Input the matrix elements
  for i in range(rows):
    row = list(map(int, file_object.readline().split()))
    matrix.append(row)
  # Display the matrix
  # print("The matrix contained in the file ",file_name," is: ")
  # for row in matrix:
  #  print(row)
  return matrix,size

# Convert an integer into a set of nodes
def convert_from_int_to_set(integer,size):
  set_of_nodes = []
  mask = 1
  for i in range(size):
    if ((mask & integer) != 0):
      set_of_nodes.append(i)
    mask = mask * 2
  return set_of_nodes

# Find the maximum independent set
def find_max_ind_set(adj_mat_numpy,size):
  max_independent_set_size = 0
  max_independent_set = []

  size_of_power_set = 1
  for i in range(size):
    size_of_power_set *= 2
  # print("The power set has ",size_of_power_set," elements")
  array_with_sizes = np.zeros(size_of_power_set)
  for i in range(size_of_power_set):
    this_set = convert_from_int_to_set(i,size)
    is_independent = True
  #  print(this_set)
  # Your code goes here:
  # For every pair of elements in this_set, check if there is an edge between them
  # If there is an edge, this_set is not an independent set
  # If there are none, this_set is an independent set.
  # Compare its size with the largest independent set found so far
  # and if it is larger, update the largest independent set and its size
    for n1 in this_set:
      for n2 in this_set:
        if (adj_mat_numpy[n1][n2] == 1):
          is_independent = False
    if (is_independent):
      array_with_sizes[i] = len(this_set)
    else:
      array_with_sizes[i] = 0


  max_independent_set_size = np.max(array_with_sizes)
  max_independent_set = np.where(array_with_sizes == max_independent_set_size)[0]
  print("The max independent sets are encoded by: ",max_independent_set)
  return max_independent_set_size



if __name__ == "__main__":
# Read the content of the file with the a passed in the command line
# that contain the matrices to be multiplied
  adj_matrix,size = read_adjacency_matrix(sys.argv[1])
  adj_matrix_numpy = np.array(adj_matrix)
  start_time = time.time()
  max_independent_set_size = find_max_ind_set(adj_matrix_numpy,size)
  end_time = time.time()
  elapsed_time = end_time - start_time
  print("Time required to carry out the computation in python: ",elapsed_time)
  print("The size of the maximum independent set is: ",max_independent_set_size)

Overwriting original_python.py


The cell below contains the file that you will modify.

Insert, just before the line

 def convert_from_int_to_set(integer,size):

 the following directive:

 @numba.jit(nopython=True)

 This directs numba to compile this function into binary code.

 Insert, just before the line
  
def find_max_ind_set(adj_mat_numpy,size):

the following directive:

 @numba.jit(nopython=True,parallel=True)

 This, again, directs numba to compile this function into binary code.

 Replace the line
  for i in range(size_of_power_set):

  with
   for i in prange(size_of_power_set):

   This instructs the numba compiler to parallelize this for statement.
   Several threads will be started to divide the execution of this for statement
   across diferent cores in the microprocessor.

   If you are curious you can perform the performance comparision on an EOS machine or on your personal computer. With more cores, you should see better improvements in execution time.

In [46]:
%%writefile with_numba.py
import sys
import time
import numpy as np
import numba

def read_adjacency_matrix(file_name):
  file_object = open(file_name, "r")
  # Input the number of rows and columns
  size = int(file_object.readline())
  rows = size
  cols = size
  # Initialize an empty matrix
  matrix = []

  # Input the matrix elements
  for i in range(rows):
    row = list(map(int, file_object.readline().split()))
    matrix.append(row)
  # Display the matrix
  # print("The matrix contained in the file ",file_name," is: ")
  # for row in matrix:
  #  print(row)
  return matrix,size

# Convert an integer into a set of nodes

def convert_from_int_to_set(integer,size):
  set_of_nodes = []
  mask = 1
  for i in range(size):
    if ((mask & integer) != 0):
      set_of_nodes.append(i)
    mask = mask * 2
  return set_of_nodes

# Find the maximum independent set

def find_max_ind_set(adj_mat_numpy,size):
  max_independent_set_size = 0
  max_independent_set = [0]

  size_of_power_set = 1
  for i in range(size):
    size_of_power_set *= 2
  # print("The power set has ",size_of_power_set," elements")
  array_with_sizes = np.zeros(size_of_power_set)
  for i in range(size_of_power_set):
    this_set = convert_from_int_to_set(i,size)
    is_independent = True
  #  print(this_set)
  # Your code goes here:
  # For every pair of elements in this_set, check if there is an edge between them
  # If there is an edge, this_set is not an independent set
  # If there are none, this_set is an independent set.
  # Compare its size with the largest independent set found so far
  # and if it is larger, update the largest independent set and its size
    for n1 in this_set:
      for n2 in this_set:
        if (adj_mat_numpy[n1][n2] == 1):
          is_independent = False
    if (is_independent):
      array_with_sizes[i] = len(this_set)
    else:
      array_with_sizes[i] = 0


  max_independent_set_size = np.max(array_with_sizes)
  max_independent_set = np.where(array_with_sizes == max_independent_set_size)[0]
  print("The max independent sets are encoded by: ",max_independent_set)
  return max_independent_set_size



if __name__ == "__main__":
# Read the content of the file with the a passed in the command line
# that contain the matrices to be multiplied
  adj_matrix,size = read_adjacency_matrix(sys.argv[1])
  adj_matrix_numpy = np.array(adj_matrix)
  # A first call to give numba the time to compile
  start_time = time.time()
  max_independent_set_size = find_max_ind_set(adj_matrix_numpy,size)
  end_time = time.time()
  elapsed_time = end_time - start_time
  print("Time required to carry out the computation with compilation: ",elapsed_time)
  # A second call to measure raw execution time, without compilation time
  start_time = time.time()
  max_independent_set_size = find_max_ind_set(adj_matrix_numpy,size)
  end_time = time.time()
  elapsed_time = end_time - start_time
  print("Time required to carry out the computation without compilation: ",elapsed_time)
  print("The size of the maximum independent set is: ",max_independent_set_size)

Overwriting with_numba.py


Now let's compare the execution times with the different test files.

In [47]:
!python original_python.py k4.txt
!python with_numba.py k4.txt
!python original_python.py no_edges_4.txt
!python with_numba.py no_edges_4.txt
!python original_python.py k16.txt
!python with_numba.py k16.txt
!python original_python.py k20.txt
!python with_numba.py k20.txt
!python with_numba.py k24.txt

The max independent sets are encoded by:  [1 2 4 8]
Time required to carry out the computation in python:  0.0003368854522705078
The size of the maximum independent set is:  1.0
The max independent sets are encoded by:  [1 2 4 8]
Time required to carry out the computation with compilation:  0.00028586387634277344
The max independent sets are encoded by:  [1 2 4 8]
Time required to carry out the computation without compilation:  0.00019741058349609375
The size of the maximum independent set is:  1.0
The max independent sets are encoded by:  [15]
Time required to carry out the computation in python:  0.0002739429473876953
The size of the maximum independent set is:  4.0
The max independent sets are encoded by:  [15]
Time required to carry out the computation with compilation:  0.00043892860412597656
The max independent sets are encoded by:  [15]
Time required to carry out the computation without compilation:  0.00026297569274902344
The size of the maximum independent set is:  4.0
The max

To summarize, enter the results of the execution times in the table below:

| Test File      | Sequential Python Execution | With Numba including compilation | With numba excluding compilation |
| ------------- | ------------- | ------------| ---------- |
| K4            |  0.0          | 0.0         |  0.0       |
| No edges 4    |  0.0          | 0.0         |  0.0       |
| K16           |  0.0          | 0.0         |  0.0       |
| K20           |  0.0          | 0.0         |  0.0       |
| K24           |  Too long...  | 0.0         |  0.0       |

